In [ ]:
!pip install rasterio

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Abrindo a Imagem


In [ ]:
# Caminho para o arquivo .tif
caminho_da_imagem = '/content/drive/MyDrive/PDI/Landsat8_Thermal_Nordeste-0000000000-0000000000.tif'

# Abrir o arquivo .tif com rasterio
with rasterio.open(caminho_da_imagem) as src:
    # Exibir a imagem
    show(src)
    plt.show()

### Visualizar a banda da imagem


In [ ]:
# Abrir o arquivo .tif com rasterio
with rasterio.open(caminho_da_imagem) as src:
    # Obter o número de bandas
    num_bands = src.count
    print(f'A imagem tem {num_bands} bandas.')

    # Visualizar cada banda separadamente
    for band in range(1, num_bands + 1):  # As bandas são indexadas a partir de 1
        # Ler a banda
        band_data = src.read(band)

        # Plotar a banda
        plt.figure(figsize=(6, 6))
        plt.imshow(band_data, cmap='gray')
        plt.title(f'Banda {band}')
        plt.axis('off')  # Remove os eixos para uma visualização mais limpa
        plt.show()

Visualizando valores de radiança e metadados da imagem

---



In [ ]:
# Abrir a imagem usando rasterio
with rasterio.open(caminho_da_imagem) as src:
    # Ler a primeira banda (supondo que seja a banda térmica)
    radiancia = src.read(1)  # Lê a banda 1 da imagem

    # Exibir estatísticas da radiança para entender a faixa de valores
    print(f"Estatísticas da Radiança:")
    print(f"Valor mínimo: {np.nanmin(radiancia)}")
    print(f"Valor máximo: {np.nanmax(radiancia)}")
    print(f"Valor médio: {np.nanmean(radiancia)}")

    # Verificar quantos valores são menores ou iguais a zero
    valores_invalidos = np.sum(radiancia <= 0)
    print(f"Número de valores inválidos (<= 0): {valores_invalidos}")

In [ ]:
# Abrir a imagem e exibir os metadados
with rasterio.open(caminho_da_imagem) as src:
    # Exibir os metadados da imagem
    print("Metadados da imagem:")
    print(src.meta)

    # Verificar se há um valor de preenchimento (nodata) definido
    if src.nodata is not None:
        print(f"Valor de preenchimento (nodata): {src.nodata}")
    else:
        print("Nenhum valor de preenchimento (nodata) definido.")

Normalizando as radiancias

In [ ]:
# Fator de escala típico para a banda térmica do Landsat 8
escala_radiancia = 0.0003342

# Abrir a imagem usando rasterio
with rasterio.open(caminho_da_imagem) as src:
    # Ler a primeira banda (supondo que seja a banda térmica)
    radiancia = src.read(1)  # Lê a banda 1 da imagem

    # Exibir estatísticas da radiança antes do ajuste
    print(f"Estatísticas da Radiança original:")
    print(f"Valor mínimo: {np.nanmin(radiancia)}")
    print(f"Valor máximo: {np.nanmax(radiancia)}")
    print(f"Valor médio: {np.nanmean(radiancia)}")

    # Aplicar o fator de escala à radiança
    radiancia_ajustada = radiancia * escala_radiancia

    # Exibir estatísticas da radiança ajustada
    print(f"Estatísticas da Radiança ajustada (após aplicar escala):")
    print(f"Valor mínimo: {np.nanmin(radiancia_ajustada)}")
    print(f"Valor máximo: {np.nanmax(radiancia_ajustada)}")
    print(f"Valor médio: {np.nanmean(radiancia_ajustada)}")

Verificando a temperatura

In [ ]:
# Constantes do Landsat 8 para a Banda 10
K1 = 774.8853  # W/(m² sr µm)
K2 = 1321.0789  # K
c2 = 1488  # µm·K (constante de Planck modificada)
lambda_banda = 10.8  # µm (comprimento de onda para a banda 10)
emissividade = 0.98  # Ajuste conforme necessário

# Fator de escala típico para a banda térmica do Landsat 8
escala_radiancia = 0.0003342

# Abrir a imagem usando rasterio
with rasterio.open(caminho_da_imagem) as src:
    # Ler a primeira banda (supondo que seja a banda térmica)
    radiancia = src.read(1)  # Lê a banda 1 da imagem

    # Aplicar o fator de escala à radiança
    radiancia_ajustada = radiancia * escala_radiancia

    # Substituir valores nulos (zeros ou negativos) por NaN para evitar erros de log
    radiancia_ajustada = np.where(radiancia_ajustada <= 0, np.nan, radiancia_ajustada)

    # Calcular a Temperatura de Brilho (T_b) para cada pixel da imagem, ignorando NaNs
    T_b = K2 / np.log((K1 / radiancia_ajustada) + 1)

    # Calcular a Temperatura de Superfície (T) em Kelvin para cada pixel da imagem, ignorando NaNs
    T_kelvin = T_b / (1 + ((lambda_banda * T_b) / c2) * np.log(emissividade))

    # Converter para Celsius
    T_celsius = T_kelvin - 273.15

    # Calcular a média ignorando NaNs
    T_kelvin_mean = np.nanmean(T_kelvin)
    T_celsius_mean = np.nanmean(T_celsius)

    # Mostrar o valor da temperatura de superfície em Kelvin
    print(f'Temperatura média de Superfície (Kelvin) para a imagem: {T_kelvin_mean:.2f} K')

    # Mostrar o valor da temperatura de superfície em Celsius
    print(f'Temperatura média de Superfície (Celsius) para a imagem: {T_celsius_mean:.2f} °C')